In [1]:
from components.ear.ear import Ear

ear = Ear()
ear.listen()

Adjusting for ambient noise...
Listening...
Speech detected, processing...
Waiting for text from whisper-1...
Listening...
***********Recognized: 시청해주셔서 감사합니다! 구독 좋아요 댓글 부탁드려요!
{'received_message': '시청해주셔서 감사합니다! 구독 좋아요 댓글 부탁드려요!'}
Speech detected, processing...
Listening...
Waiting for text from whisper-1...
***********Recognized: Hello.
{'received_message': 'Hello.'}
Speech detected, processing...
Skipping processing. 5.7 seconds until next process.
Listening...
Speech detected, processing...
Skipping processing. 3.1 seconds until next process.
Listening...
Speech detected, processing...
Skipping processing. 1.5 seconds until next process.
Listening...
Speech detected, processing...
Listening...
Waiting for text from whisper-1...
***********Recognized: you
{'received_message': 'you'}
Speech detected, processing...
Skipping processing. 6.0 seconds until next process.
Listening...
Speech detected, processing...
Listening...
Waiting for text from whisper-1...
***********Recognized: Ha.
